1.Answer : 
 We will solve the optimization problem:
Max 2*x1+3*x2+5*x3+4*x4+6*x5+6*x6+8*x7+8*x8+6*x9+4*x10+6*x11,
s.t. 
2*x1+3*x2+5*x3+4*x4+6*x5+6*x6+8*x7+8*x8+6*x9+4*x10+6*x11 >=26,
5 <= x1+x2+x3+x4+x5+x6+x7+x8+x9+x10+x11 <= 9 , 
x6 + x8 <= 1 , x4 + x10 <= 1 , x3 + x8 <=1,
x11 - x2 <= 0 , 1<= x1+x2 <= 1 ,x3+x5<=1, x9+x11 <= 1  .
where x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11 ∈{0,1}



In [ ]:
!pip install -q pyomo

In [ ]:
from pyomo.environ import *

In [ ]:
import numpy as np

In [ ]:
model_milp = ConcreteModel()

In [ ]:
N = 11
M = 11
lb = np.array([0,0,0,0,0,0,0,0,0,0,0])
ub = np.array([1,1,1,1,1,1,1,1,1,1,1]) 

In [ ]:
obj_coef_c = np.array([2,3,5,4,6,6,8,8,6,4,6]) 

In [ ]:
A=np.array([[-2,-3,-5,-4,-6,-6,-8,-8,-6,-4,-6],[1,1,1,1,1,1,1,1,1,1,1],[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
           [0,0,0,0,0,1,0,1,0,0,0],[0,0,0,1,0,0,0,0,0,1,0],
           [0,0,1,0,0,0,0,1,0,0,0],[0,-1,0,0,0,0,0,0,0,0,1],[1,1,0,0,0,0,0,0,0,0,0],
           [-1,-1,0,0,0,0,0,0,0,0,0],[0,0,1,0,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,1,0,1]])
B=np.array([-26,9,-5,1,1,1,0,1,-1,1,1])
row_indices=np.arange(M)
print(row_indices)
col_indices = np.arange(N)
print(col_indices)

[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]


In [ ]:
model_milp.x = Var(col_indices,domain=Binary)
model_milp.constraints = ConstraintList()

In [ ]:
for i in col_indices:
  model_milp.x[i].setlb(lb[i])
  model_milp.x[i].setub(ub[i])

In [ ]:
for i in row_indices:
  model_milp.constraints.add(sum(A[i][j]*model_milp.x[j] for j in col_indices) <= B[i])

In [ ]:
model_milp.objective = Objective(expr =summation(obj_coef_c,model_milp.x), sense=maximize)
model_milp.pprint() 

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None 

In [ ]:
!apt-get install -y -qq coinor-cbc

In [ ]:
opt_cbc = SolverFactory('cbc')

In [ ]:
result = opt_cbc.solve(model_milp)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 35.0
  Upper bound: 35.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 11
  Number of binary variables: 11
  Number of integer variables: 11
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.028574228286743164
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [ ]:
# display solution
print('\nObjective = ', model_milp.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_milp.x[i].value)
print('\nActive Constraints')
model_milp.constraints.display()


Objective =  35.0

Decision Variables
x[ 0 ] =  0.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  0.0
x[ 9 ] =  1.0
x[ 10 ] =  1.0

Active Constraints
constraints : Size=11
    Key : Lower : Body  : Upper
      1 :  None : -35.0 : -26.0
      2 :  None :   6.0 :   9.0
      3 :  None :  -6.0 :  -5.0
      4 :  None :   1.0 :   1.0
      5 :  None :   1.0 :   1.0
      6 :  None :   1.0 :   1.0
      7 :  None :   0.0 :   0.0
      8 :  None :   1.0 :   1.0
      9 :  None :  -1.0 :  -1.0
     10 :  None :   1.0 :   1.0
     11 :  None :   1.0 :   1.0


4. Now we are removing the restrictions that
the variables in the above problem are integers

In [ ]:
model_milp_1 = ConcreteModel()

In [ ]:
N = 11
M = 11
lb = np.array([0,0,0,0,0,0,0,0,0,0,0])
ub = np.array([1,1,1,1,1,1,1,1,1,1,1]) 

In [ ]:
obj_coef_c = np.array([2,3,5,4,6,6,8,8,6,4,6]) 

In [ ]:
A=np.array([[-2,-3,-5,-4,-6,-6,-8,-8,-6,-4,-6],[1,1,1,1,1,1,1,1,1,1,1],[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
           [0,0,0,0,0,1,0,1,0,0,0],[0,0,0,1,0,0,0,0,0,1,0],
           [0,0,1,0,0,0,0,1,0,0,0],[0,-1,0,0,0,0,0,0,0,0,1],[1,1,0,0,0,0,0,0,0,0,0],
           [-1,-1,0,0,0,0,0,0,0,0,0],[0,0,1,0,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,1,0,1]])
B=np.array([-26,9,-5,1,1,1,0,1,-1,1,1])
row_indices=np.arange(M)
print(row_indices)
col_indices = np.arange(N)
print(col_indices)

[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]


In [ ]:
#here we have removed the restriction on domain which was earlier Binary
model_milp_1.x = Var(col_indices)
model_milp_1.constraints = ConstraintList()

In [ ]:
for i in col_indices:
  model_milp_1.x[i].setlb(lb[i])
  model_milp_1.x[i].setub(ub[i])

In [ ]:
for i in row_indices:
  model_milp_1.constraints.add(sum(A[i][j]*model_milp_1.x[j] for j in col_indices) <= B[i])

In [ ]:
model_milp_1.objective = Objective(expr =summation(obj_coef_c,model_milp_1.x), sense=maximize)
model_milp_1.pprint() 

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True :  Reals
          1 :     0 :  None :     1 : False :  True :  Reals
          2 :     0 :  None :     1 : False :  True :  Reals
          3 :     0 :  None :     1 : False :  True :  Reals
          4 :     0 :  None :     1 : False :  True :  Reals
          5 :     0 :  None :     1 : False :  True :  Reals
          6 :     0 :  None :     1 : False :  True :  Reals
          7 :     0 :  None :     1 : False :  True :  Reals
          8 :     0 :  None 

In [ ]:
!apt-get install -y -qq coinor-cbc

In [ ]:
opt_cbc_1 = SolverFactory('cbc')

In [ ]:
result = opt_cbc_1.solve(model_milp_1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 35.0
  Upper bound: 35.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 12
  Number of nonzeros: 10
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 4
  Error rc: 0
  Time: 0.02390456199645996
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [ ]:
# display solution
print('\nObjective = ', model_milp_1.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_milp_1.x[i].value)
print('\nActive Constraints')
model_milp_1.constraints.display()


Objective =  35.0

Decision Variables
x[ 0 ] =  0.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  1.0
x[ 10 ] =  0.0

Active Constraints
constraints : Size=11
    Key : Lower : Body  : Upper
      1 :  None : -35.0 : -26.0
      2 :  None :   6.0 :   9.0
      3 :  None :  -6.0 :  -5.0
      4 :  None :   1.0 :   1.0
      5 :  None :   1.0 :   1.0
      6 :  None :   1.0 :   1.0
      7 :  None :  -1.0 :   0.0
      8 :  None :   1.0 :   1.0
      9 :  None :  -1.0 :  -1.0
     10 :  None :   1.0 :   1.0
     11 :  None :   1.0 :   1.0


5.Answer: No its not possible. I guess(not sure),rounding solutions can cause the solution to become infeasible.

6.Answer:
Here we are required to add a new decision variable x12 ∈{0,1} , with new_objective: Max 2*x1+3*x2+5*x3+4*x4+6*x5+6*x6+8*x7+8*x8+6**x9+4*x10+6*x11+9*x12 and adding a new_constraint: x8+x9+x12 <=1 to the original optimization problem mentioned at the top.

In [ ]:
N = 12
model_milp.x12 = Var(domain=Binary) 

In [ ]:
model_milp.constraints.add(expr = model_milp.x[7]+ model_milp.x[8]+model_milp.x12 <= 1)

In [ ]:
#deactivate the old objective
model_milp.objective.deactivate()

In [ ]:
model_milp.new_objective = Objective(expr = 9*model_milp.x12 + summation(obj_coef_c,model_milp.x),sense=maximize )

In [ ]:
model_milp.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

2 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False : Binary
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     0 :   0.0 :     1 : False : False : Binary
          6 :     0 :   1.0 :     1 : False : False : Binary
          7 :     0 :   1.0 :     1 : False : False : Binary
          8 :     0 :   

In [ ]:
!apt-get install -y -qq coinor-cbc

In [ ]:
opt_cbc = SolverFactory('cbc')

In [ ]:
result = opt_cbc.solve(model_milp)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 42.0
  Upper bound: 42.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 12
  Number of binary variables: 12
  Number of integer variables: 12
  Number of nonzeros: 12
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.024747133255004883
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [ ]:
# display solution
print('\nObjective = ', model_milp.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_milp.x[i].value)
print('\nActive Constraints')
model_milp.constraints.display()


Objective =  33.0

Decision Variables
x[ 0 ] =  0.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  1.0
x[ 5 ] =  1.0
x[ 6 ] =  1.0
x[ 7 ] =  0.0
x[ 8 ] =  0.0
x[ 9 ] =  1.0
x[ 10 ] =  1.0

Active Constraints
constraints : Size=12
    Key : Lower : Body  : Upper
      1 :  None : -33.0 : -26.0
      2 :  None :   6.0 :   9.0
      3 :  None :  -6.0 :  -5.0
      4 :  None :   1.0 :   1.0
      5 :  None :   1.0 :   1.0
      6 :  None :   0.0 :   1.0
      7 :  None :   0.0 :   0.0
      8 :  None :   1.0 :   1.0
      9 :  None :  -1.0 :  -1.0
     10 :  None :   1.0 :   1.0
     11 :  None :   1.0 :   1.0
     12 :  None :   1.0 :   1.0


8.Now we will remove the integer restriction

In [ ]:
model_milp = ConcreteModel()

In [ ]:
N = 12
M = 12
lb = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
ub = np.array([1,1,1,1,1,1,1,1,1,1,1,1]) 

In [ ]:
obj_coef_c = np.array([2,3,5,4,6,6,8,8,6,4,6,9]) 

In [ ]:
A=np.array([[-2,-3,-5,-4,-6,-6,-8,-8,-6,-4,-6,0],[1,1,1,1,1,1,1,1,1,1,1,0],[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0],
           [0,0,0,0,0,1,0,1,0,0,0,0],[0,0,0,1,0,0,0,0,0,1,0,0],
           [0,0,1,0,0,0,0,1,0,0,0,0],[0,-1,0,0,0,0,0,0,0,0,1,0],[1,1,0,0,0,0,0,0,0,0,0,0],
           [-1,-1,0,0,0,0,0,0,0,0,0,0],[0,0,1,0,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,1,0,1,0],[0,0,0,0,0,0,0,1,1,0,0,1]])
B=np.array([-26,9,-5,1,1,1,0,1,-1,1,1,1])
row_indices=np.arange(M)
print(row_indices)
col_indices = np.arange(N)
print(col_indices)

[ 0  1  2  3  4  5  6  7  8  9 10 11]
[ 0  1  2  3  4  5  6  7  8  9 10 11]


In [ ]:
#here we have removed the restriction on domain which was earlier Binary
model_milp.x = Var(col_indices)
model_milp.constraints = ConstraintList()

In [ ]:
for i in col_indices:
  model_milp.x[i].setlb(lb[i])
  model_milp.x[i].setub(ub[i])

In [ ]:
for i in row_indices:
  model_milp.constraints.add(sum(A[i][j]*model_milp.x[j] for j in col_indices) <= B[i])

In [ ]:
model_milp.objective = Objective(expr =summation(obj_coef_c,model_milp.x), sense=maximize)
model_milp.pprint() 

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    x : Size=12, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True :  Reals
          1 :     0 :  None :     1 : False :  True :  Reals
          2 :     0 :  None :     1 : False :  True :  Reals
          3 :     0 :  None :     1 : False :  True :  Reals
          4 :     0 :  None :     1 : False :  True :  Reals
          5 :     0 :  None :     1 : False :  True :  Reals
          6 :     0 :  None :     1 : False :  True :  Reals
          7 :     0 :  None :     1 : False :  True :  Reals
          8 :     0 

In [ ]:
!apt-get install -y -qq coinor-cbc

In [ ]:
opt_cbc = SolverFactory('cbc')

In [ ]:
result = opt_cbc.solve(model_milp)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 42.0
  Upper bound: 42.0
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 13
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 3
  Error rc: 0
  Time: 0.02772974967956543
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [ ]:
# display solution
print('\nObjective = ', model_milp.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_milp.x[i].value)
print('\nActive Constraints')
model_milp.constraints.display()


Objective =  42.0

Decision Variables
x[ 0 ] =  0.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  1.0
x[ 5 ] =  1.0
x[ 6 ] =  1.0
x[ 7 ] =  0.0
x[ 8 ] =  0.0
x[ 9 ] =  1.0
x[ 10 ] =  1.0
x[ 11 ] =  1.0

Active Constraints
constraints : Size=12
    Key : Lower : Body  : Upper
      1 :  None : -33.0 : -26.0
      2 :  None :   6.0 :   9.0
      3 :  None :  -6.0 :  -5.0
      4 :  None :   1.0 :   1.0
      5 :  None :   1.0 :   1.0
      6 :  None :   0.0 :   1.0
      7 :  None :   0.0 :   0.0
      8 :  None :   1.0 :   1.0
      9 :  None :  -1.0 :  -1.0
     10 :  None :   1.0 :   1.0
     11 :  None :   1.0 :   1.0
     12 :  None :   1.0 :   1.0
